# Register Number: 8569202378

# Name: Ashvant Ram Selvam

# Question 2

#### (a) Download the Anuran Calls (MFCCs) Data Set from: https://archive.ics.uci.edu/ml/datasets/Anuran+Calls+%28MFCCs%29#. Choose 70% of the data randomly as the training set.

In [2]:
import pandas as pd
import numpy as np
fileData = pd.read_csv("Frogs_MFCCs.csv")
print(fileData)

      MFCCs_ 1  MFCCs_ 2  MFCCs_ 3  MFCCs_ 4  MFCCs_ 5  MFCCs_ 6  MFCCs_ 7  \
0          1.0  0.152936 -0.105586  0.200722  0.317201  0.260764  0.100945   
1          1.0  0.171534 -0.098975  0.268425  0.338672  0.268353  0.060835   
2          1.0  0.152317 -0.082973  0.287128  0.276014  0.189867  0.008714   
3          1.0  0.224392  0.118985  0.329432  0.372088  0.361005  0.015501   
4          1.0  0.087817 -0.068345  0.306967  0.330923  0.249144  0.006884   
5          1.0  0.099704 -0.033408  0.349895  0.344535  0.247569  0.022407   
6          1.0  0.021676 -0.062075  0.318229  0.380439  0.179043 -0.041667   
7          1.0  0.145130 -0.033660  0.284166  0.279537  0.175211  0.005791   
8          1.0  0.271326  0.027777  0.375738  0.385432  0.272457  0.098192   
9          1.0  0.120565 -0.107235  0.316555  0.364437  0.307757  0.025992   
10         1.0  0.148539 -0.096910  0.257523  0.260881  0.312603  0.134134   
11         1.0  0.277948  0.091657  0.331656  0.307372  0.257359

In [10]:
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold, KFold, GridSearchCV
train, test = train_test_split(fileData, test_size=.3)
trainLabel = pd.DataFrame(train[["Family","Species","Genus"]])
trainData = train.drop(["Family", "Genus", "Species", "RecordID"], axis=1)
testLabel = pd.DataFrame(test[["Family","Species","Genus"]])
testData = test.drop(["Family", "Genus", "Species", "RecordID"], axis=1)

(b) Each instance has three labels: Families, Genus, and Species. Each of the labels
has multiple classes. We wish to solve a multi-class and multi-label problem.
One of the most important approaches to multi-class classification is to train a
classifier for each label. We first try this approach:

i. Research exact match and hamming score/ loss methods for evaluating multilabel
classification and use them in evaluating the classifiers in this problem.

Exact Match:
Exact Match loss is a very strict parameter which considers the sample correctly classified only if all the three labels are classified correctly. Any partial correct answer is considered as wrong classification.

Hamming Loss: Hamming loss is a less strict parameter which takes into account partial correct answers. Hence, hamming loss usually gives a lower value error value compared to exact match loss as partial correct answers are taken into account and not completely classified as wrong classification.

ii. Train a SVM for each of the labels, using Gaussian kernels and one versus all classifiers. Determine the weight of the SVM penalty and the width of the Gaussian Kernel using 10 fold cross validation. You are welcome to try to solve the problem with both normalized and raw attributes and report the
results.

Normalize the features

In [55]:
from sklearn.preprocessing import normalize
trainDataNonormalize = trainData
trainData = normalize(trainData)

In [26]:
from sklearn.svm import LinearSVC, SVC
from sklearn.multiclass import OneVsRestClassifier
def fit_gaussian(trainDataGaussian, trainLabelGaussian,classEstimated):
    sig = np.array([value for value in range(1,20,2)])/10
    width = 1/(2*sig*sig)
    params = {'estimator__C':(10**np.array([value for value in range(0,6)]))/100, 'estimator__gamma':width}
    print(classEstimated)
    svc = SVC()
    classifier = GridSearchCV(OneVsRestClassifier(svc), params, cv=10)
    classifier.fit(trainDataGaussian, trainLabelGaussian)
    print("Best Weight of SVM penalty   = ", classifier.best_params_['estimator__C'])
    print("Best Width of Gaussian Kernel   = ", classifier.best_params_['estimator__gamma'])
    return classifier

In [27]:
svc_family = fit_gaussian(trainData, trainLabel["Family"],"Family")
svc_genus = fit_gaussian(trainData, trainLabel["Genus"],"Genus")
svc_species = fit_gaussian(trainData, trainLabel["Species"],"Species")

Family
Best Weight of SVM penalty   =  1000.0
Best Width of Gaussian Kernel   =  2.0
Genus
Best Weight of SVM penalty   =  10.0
Best Width of Gaussian Kernel   =  5.55555555556
Species
Best Weight of SVM penalty   =  10.0
Best Width of Gaussian Kernel   =  2.0


Above are the cross validated parameters obtained for each of the svms's constructed for the specific class labels. These SVMs will be used to calculate the hamming and exact match loss for the normalized train data.

In [28]:
def hamming_loss(trueLabels,predictedLabels):
    total=0
    for trueRow,predictedRow in zip(trueLabels,predictedLabels):
        correct=0
        for a,b in zip(trueRow,predictedRow):
            if(a==b):
                correct+=1
        ratioOfCorrect = correct/len(trueRow)
        total+=ratioOfCorrect
    return total/len(trueLabels)        

In [29]:
def exactMatch_loss(trueLabels,predictedLabels):
    total=0
    for trueRow,predictedRow in zip(trueLabels,predictedLabels):
        correct=1
        for a,b in zip(trueRow,predictedRow):
            if(a!=b):
                correct=0
        total+=correct
    return total/len(trueLabels)

In [34]:
predicted = np.array([svc_family.predict(testData), svc_species.predict(testData), svc_genus.predict(testData)]).T
print("Hamming Loss : ", 1-hamming_loss(np.array(testLabel), predicted))
print("Exact Match Loss     : ", 1-exactMatch_loss(np.array(testLabel), predicted))

Hamming Loss :  0.010189902732746225
Exact Match Loss     :  0.016674386289949106


As expected the exact match loss is higher compared to hamming loss since exact match loss is strict measure which considers partially correct classifications as wrong classifications while calculating accuracy.

One Vs ALL Classifier

In [39]:
def fit_linear_OneVsRest(trainDataLinearOvR, trainLabelLinearOvR,classEstimated):
    params = {'estimator__C':(10**np.array([value for value in range(0,6)]))/100}
    print(classEstimated)
    svc = LinearSVC(penalty='l1', dual = False)
    classifier = GridSearchCV(OneVsRestClassifier(svc, n_jobs=-1), params, cv=10)
    classifier.fit(trainDataLinearOvR, trainLabelLinearOvR)
    print("Best Weight of SVM penalty   = ", classifier.best_params_['estimator__C'])
    return classifier

In [40]:
svc_family = fit_linear_OneVsRest(trainData, trainLabel["Family"],"Family")
svc_genus = fit_linear_OneVsRest(trainData, trainLabel["Genus"],"Genus")
svc_species = fit_linear_OneVsRest(trainData, trainLabel["Species"],"Species")

Family
Best Weight of SVM penalty   =  10.0
Genus
Best Weight of SVM penalty   =  100.0
Species
Best Weight of SVM penalty   =  100.0


In [41]:
predicted = np.array([svc_family.predict(testData), svc_species.predict(testData), svc_genus.predict(testData)]).T
print("Hamming Loss : ", 1-hamming_loss(np.array(testLabel), predicted))
print("Exact Match Loss     : ", 1-exactMatch_loss(np.array(testLabel), predicted))

Hamming Loss :  0.054809325304924994
Exact Match Loss     :  0.08661417322834641


iii. Repeat 2(b)ii with L1-penalized SVMs. Remember to normalize the attributes.

The attributes are already normalized in the data preprocessing step.

In [42]:
def fit_linear(trainDataLinear, trainLabelLinear,classEstimated):
    params = {'C':(10**np.array([value for value in range(0,6)]))/100}
    print(classEstimated)
    svc = LinearSVC(penalty='l1', dual = False)
    classifier = GridSearchCV(svc, params, cv=10)
    classifier.fit(trainDataLinear, trainLabelLinear)
    print("Best Weight of SVM penalty   = ", classifier.best_params_['C'])
    return classifier

In [43]:
svc_family = fit_linear(trainData, trainLabel["Family"],"Family")
svc_genus = fit_linear(trainData, trainLabel["Genus"],"Genus")
svc_species = fit_linear(trainData, trainLabel["Species"],"Species")

Family
Best Weight of SVM penalty   =  100.0
Genus
Best Weight of SVM penalty   =  100.0
Species
Best Weight of SVM penalty   =  1000.0


In [56]:
predicted = np.array([svc_family.predict(testData), svc_species.predict(testData), svc_genus.predict(testData)]).T
print("Exact Match Loss : ", 1-exactMatch_loss(np.array(testLabel), predicted))
print("Hamming Loss     : ", 1-hamming_loss(np.array(testLabel), predicted))

Exact Match Loss : 0.08707735062528954
Hamming Loss     : 0.05496371777057263


iv. Repeat 2(b)iii by using SMOTE or any other method you know to remedy
class imbalance. Report your conclusions about the classifiers you trained.

In [46]:
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
trainDataSMOTE_family, trainLabelSMOTE_family = sm.fit_sample(trainData, trainLabel["Family"])
sm = SMOTE(random_state=42)
trainDataSMOTE_genus, trainLabelSMOTE_genus = sm.fit_sample(trainData, trainLabel["Genus"])
sm = SMOTE(random_state=42)
trainDataSMOTE_species, trainLabelSMOTE_species = sm.fit_sample(trainData, trainLabel["Species"])

In [51]:
svc_family = fit_gaussian(trainDataSMOTE_family, trainLabelSMOTE_family, "Family")
svc_genus = fit_gaussian(trainDataSMOTE_genus, trainLabelSMOTE_genus, "Genus")
svc_species = fit_gaussian(trainDataSMOTE_species, trainLabelSMOTE_species, "Species")

Family
Best Weight of SVM penalty   = 10
Best Width of Gaussian Kernel   = 2.0
Genus
Best Weight of SVM penalty   = 100
Best Width of Gaussian Kernel   = 5.55555555556
Species
Best Weight of SVM penalty   = 10
Best Width of Gaussian Kernel   = 2.0


In [58]:
predicted = np.array([svc_family.predict(testData), svc_species.predict(testData), svc_genus.predict(testData)]).T
print("Hamming Loss     : ", 1-hamming_loss(np.array(testLabel), predicted))
print("Exact Match Loss     : ", 1-exactMatch_loss(np.array(testLabel), predicted))

Hamming Loss : 0.00812783696155708
Exact Match Loss     : 0.01345530338119498


In [52]:
svc_family = fit_linear(trainDataSMOTE_family, trainLabelSMOTE_family, "Family")
svc_genus = fit_linear(trainDataSMOTE_genus, trainLabelSMOTE_genus, "Genus")
svc_species = fit_linear(trainDataSMOTE_species, trainLabelSMOTE_species, "Species")

Family
Best Weight of SVM penalty   = 1000
Genus
Best Weight of SVM penalty   = 10
Species
Best Weight of SVM penalty   = 100


In [62]:
predicted = np.array([svc_family.predict(testData), svc_species.predict(testData), svc_genus.predict(testData)]).T
print("Exact Match Loss : ", 1-exactMatch_loss(np.array(testLabel), predicted))
print("Hamming Loss     : ", 1-hamming_loss(np.array(testLabel), predicted))

Exact Match Loss : 0.034861047284797155
Hamming Loss     : 0.01992665274720124


On handling imbalance using SMOTE the oversampled training data seems to reduce the exact match and hamming loss.
This evidently shows that our original data suffered from imbalance and we have effectively handled imbalance by oversampling using SMOTE.